In [10]:
import csv
import pandas
import json
import requests
import itertools

In [3]:
src_dir = "C:\\Users\\AaronJacobsonDesk3\\Documents\\UWMSDS\\Classes\\Fall2021\\DATA512\\DATA512\\DATA512\\A2\\"

In [130]:
df_page = pandas.read_csv(src_dir + "page_data.csv")
df_page = df_page[~df_page['page'].str.startswith("Template:")]
df_page

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


In [156]:
df_country = pandas.read_csv(src_dir + "WPDS_2020_data - WPDS_2020_data.csv")
df_region = df_country[df_country['Name'] == df_country['Name'].str.upper()]
df_country = df_country[df_country['Name'] != df_country['Name'].str.upper()]
df_country

,FIPS,Name,Type,TimeFrame,Data (M),Population
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000
5,LY,Libya,Country,2019,6.891,6891000
6,MA,Morocco,Country,2019,35.952,35952000
7,SD,Sudan,Country,2019,43.849,43849000
...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000
230,SB,Solomon Islands,Country,2019,0.715,715000
231,TO,Tonga,Country,2019,0.099,99000
232,TV,Tuvalu,Country,2019,0.010,10000


In [20]:
endpoint = "https://ores.wikimedia.org/v3/scores/enwiki?models=articlequality&revids="

def GetQuality(rev_list):
    rev_string = ""
    first_rev = True
    for revid in rev_list:
        if first_rev:
            first_rev = False
            rev_string = revid
        else:
            rev_string = rev_string + "|" + revid
    response = requests.get(endpoint + rev_string)
    body = response.json()
    return body

In [23]:
responses = []

rev_ids = []
rev_count = 0
for index, row in df_page.iterrows():
    if rev_count == 49:
        response = GetQuality(rev_ids)
        rev_count = 0
        responses.append(response)
        rev_ids = []
    else:
        rev_count += 1
        rev_ids.append(str(row['rev_id']))

In [41]:
predictions = {}
no_predictions = []
for response in responses:
    scores = response['enwiki']['scores']
    for revid in scores.keys():
        article_quality = scores[revid]['articlequality']
        if 'error' in article_quality.keys():
            no_predictions.append(revid)
        else:
            predictions[revid] = article_quality['score']['prediction']
predictions

{'355319463': 'Stub',
 '393276188': 'Stub',
 '393822005': 'Stub',
 '395521877': 'Stub',
 '395526568': 'Stub',
 '401577829': 'Stub',
 '442937236': 'Stub',
 '448555418': 'Stub',
 '470173494': 'Start',
 '477962574': 'Stub',
 '492060822': 'Stub',
 '492964343': 'Stub',
 '498683267': 'Stub',
 '502721672': 'C',
 '521986779': 'Stub',
 '532253442': 'Stub',
 '543225630': 'Stub',
 '545936100': 'Stub',
 '546364151': 'Start',
 '549300521': 'Stub',
 '550953646': 'Stub',
 '559553872': 'Stub',
 '559788982': 'Stub',
 '560758943': 'Stub',
 '561744402': 'Stub',
 '564873005': 'Stub',
 '565745353': 'Stub',
 '565745365': 'Stub',
 '565745375': 'Stub',
 '566504165': 'Stub',
 '573710096': 'B',
 '574571582': 'Stub',
 '576988466': 'Stub',
 '585894477': 'Stub',
 '592289232': 'Stub',
 '595693452': 'Start',
 '596181202': 'Start',
 '598819900': 'Start',
 '601122766': 'Stub',
 '601127343': 'Start',
 '614786300': 'Start',
 '623004627': 'Stub',
 '623334577': 'Stub',
 '624468970': 'Stub',
 '625509885': 'Start',
 '626606

In [132]:
df_preds = pandas.DataFrame.from_dict({'rev_id' : predictions.keys(), 'quality' : predictions.values()})
df_page['rev_id'] = df_page['rev_id'].astype(str)
df_page = df_page.merge(df_preds, on = 'rev_id')

In [133]:
df_no_match = df_page.merge(df_country, how='outer', left_on='country', right_on='Name')
df_no_match = df_no_match[df_no_match['Population'].isna() | df_no_match['page'].isna()]
df_no_match

,page,country,rev_id,quality,FIPS,Name,Type,TimeFrame,Data (M),Population
482,Julius Gregr,Czech Republic,395521877,Stub,NaN,NaN,NaN,NaN,NaN,NaN
483,Edvard Gregr,Czech Republic,395526568,Stub,NaN,NaN,NaN,NaN,NaN,NaN
484,Miroslav Poche,Czech Republic,672862914,Stub,NaN,NaN,NaN,NaN,NaN,NaN
485,Vojtěch Mynář,Czech Republic,673008587,Stub,NaN,NaN,NaN,NaN,NaN,NaN
486,Jan Malypetr,Czech Republic,704424304,Stub,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
45516,NaN,NaN,NaN,NaN,PF,French Polynesia,Country,2019.0,0.280,280000.0
45517,NaN,NaN,NaN,NaN,GU,Guam,Country,2019.0,0.175,175000.0
45518,NaN,NaN,NaN,NaN,NC,New Caledonia,Country,2019.0,0.295,295000.0
45519,NaN,NaN,NaN,NaN,PW,Palau,Country,2019.0,0.018,18000.0


In [134]:
df_total = df_page.merge(df_country, how='inner', left_on='country', right_on='Name')[['page','country','rev_id','quality','Population']]
df_total

,page,country,rev_id,quality,Population
0,Bir I of Kanem,Chad,355319463,Stub,16877000
1,Abdullah II of Kanem,Chad,498683267,Stub,16877000
2,Salmama II of Kanem,Chad,565745353,Stub,16877000
3,Kuri I of Kanem,Chad,565745365,Stub,16877000
4,Mohammed I of Kanem,Chad,565745375,Stub,16877000
...,...,...,...,...,...
43676,Danielle de St. Jorre,Seychelles,798671157,Stub,98000
43677,Rita Sinon,Seychelles,800323154,Stub,98000
43678,Sylvette Frichot,Seychelles,800323798,Stub,98000
43679,Vincent Meriton,Seychelles,802051093,Stub,98000


In [135]:
df_articles = df_total.groupby(['country']).count()['page']
df_articles

country
Afghanistan    313
Albania        449
Algeria        115
Andorra         33
Angola         105
              ... 
Venezuela      126
Vietnam        184
Yemen          114
Zambia          25
Zimbabwe       159
Name: page, Length: 183, dtype: int64

In [136]:
df_country = df_country.merge(df_articles, how='left', left_on='Name', right_on='country')
df_country['PagesPerCapita'] = df_country['page'] / df_country['Population'].astype(float)

In [154]:
df_country.sort_values('PagesPerCapita', ascending=False).head(10)[['Name', 'PagesPerCapita']]

,Name,PagesPerCapita
207,Tuvalu,0.005400
199,Nauru,0.004727
188,San Marino,0.002353
164,Monaco,0.001000
162,Liechtenstein,0.000692
206,Tonga,0.000636
198,Marshall Islands,0.000632
151,Iceland,0.000530
178,Andorra,0.000402
193,Federated States of Micronesia,0.000340


In [153]:
df_country.sort_values('PagesPerCapita', ascending=True).head(10)[['Name', 'PagesPerCapita']]

,Name,PagesPerCapita
141,"China, Hong Kong SAR",0.0
79,Puerto Rico,0.0
32,Mayotte,0.0
138,Timor-Leste,0.0
107,Oman,0.0
71,Curacao,0.0
53,eSwatini,0.0
34,Reunion,0.0
61,El Salvador,0.0
81,Saint Lucia,0.0


In [139]:
df_country = df_country.merge(df_total[df_total['quality'] == 'FA'].groupby(['country']).count()['page'], how='left', left_on='Name', right_on='country')
df_country

,FIPS,Name,Type,TimeFrame,Data (M),Population,page_x,PagesPerCapita,page_y
0,DZ,Algeria,Country,2019,44.357,44357000,115.0,0.000003,NaN
1,EG,Egypt,Country,2019,100.803,100803000,228.0,0.000002,6.0
2,LY,Libya,Country,2019,6.891,6891000,109.0,0.000016,1.0
3,MA,Morocco,Country,2019,35.952,35952000,203.0,0.000006,NaN
4,SD,Sudan,Country,2019,43.849,43849000,93.0,0.000002,1.0
...,...,...,...,...,...,...,...,...,...
204,WS,Samoa,Country,2019,0.200,200000,NaN,NaN,NaN
205,SB,Solomon Islands,Country,2019,0.715,715000,96.0,0.000134,NaN
206,TO,Tonga,Country,2019,0.099,99000,63.0,0.000636,NaN
207,TV,Tuvalu,Country,2019,0.010,10000,54.0,0.005400,NaN


In [140]:
df_country = df_country.merge(df_total[df_total['quality'] == 'GA'].groupby(['country']).count()['page'], how='left', left_on='Name', right_on='country')
df_country

,FIPS,Name,Type,TimeFrame,Data (M),Population,page_x,PagesPerCapita,page_y,page
0,DZ,Algeria,Country,2019,44.357,44357000,115.0,0.000003,NaN,2.0
1,EG,Egypt,Country,2019,100.803,100803000,228.0,0.000002,6.0,4.0
2,LY,Libya,Country,2019,6.891,6891000,109.0,0.000016,1.0,3.0
3,MA,Morocco,Country,2019,35.952,35952000,203.0,0.000006,NaN,1.0
4,SD,Sudan,Country,2019,43.849,43849000,93.0,0.000002,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
204,WS,Samoa,Country,2019,0.200,200000,NaN,NaN,NaN,NaN
205,SB,Solomon Islands,Country,2019,0.715,715000,96.0,0.000134,NaN,NaN
206,TO,Tonga,Country,2019,0.099,99000,63.0,0.000636,NaN,NaN
207,TV,Tuvalu,Country,2019,0.010,10000,54.0,0.005400,NaN,4.0


In [142]:
df_country = df_country.fillna(0)

In [148]:
df_country['good_articles'] = df_country['page_y'] + df_country['page']
df_country['good_articles_portion'] = df_country['good_articles'] / df_country['page_x']
df_country = df_country.fillna(0)
df_country

,FIPS,Name,Type,TimeFrame,Data (M),Population,page_x,PagesPerCapita,page_y,page,good_articles,good_articles_portion
0,DZ,Algeria,Country,2019,44.357,44357000,115.0,0.000003,0.0,2.0,2.0,0.017391
1,EG,Egypt,Country,2019,100.803,100803000,228.0,0.000002,6.0,4.0,10.0,0.043860
2,LY,Libya,Country,2019,6.891,6891000,109.0,0.000016,1.0,3.0,4.0,0.036697
3,MA,Morocco,Country,2019,35.952,35952000,203.0,0.000006,0.0,1.0,1.0,0.004926
4,SD,Sudan,Country,2019,43.849,43849000,93.0,0.000002,1.0,1.0,2.0,0.021505
...,...,...,...,...,...,...,...,...,...,...,...,...
204,WS,Samoa,Country,2019,0.200,200000,0.0,0.000000,0.0,0.0,0.0,0.000000
205,SB,Solomon Islands,Country,2019,0.715,715000,96.0,0.000134,0.0,0.0,0.0,0.000000
206,TO,Tonga,Country,2019,0.099,99000,63.0,0.000636,0.0,0.0,0.0,0.000000
207,TV,Tuvalu,Country,2019,0.010,10000,54.0,0.005400,0.0,4.0,4.0,0.074074


In [151]:
df_country.sort_values('good_articles_portion', ascending=False).head(10)[['Name', 'good_articles_portion']]

,Name,good_articles_portion
144,"Korea, North",0.222222
110,Saudi Arabia,0.133929
173,Romania,0.125000
45,Central African Republic,0.123077
119,Uzbekistan,0.107143
17,Mauritania,0.104167
62,Guatemala,0.085366
72,Dominica,0.083333
111,Syria,0.080000
7,Benin,0.077778


In [152]:
df_country.sort_values('good_articles_portion', ascending=True).head(10)[['Name','good_articles_portion']]

,Name,good_articles_portion
61,El Salvador,0.0
71,Curacao,0.0
69,Barbados,0.0
68,Bahamas,0.0
67,Antigua and Barbuda,0.0
63,Honduras,0.0
178,Andorra,0.0
60,Costa Rica,0.0
59,Belize,0.0
55,Namibia,0.0


In [ ]:


with open(src_dir + "WPDS_2020_data - WPDS_2020_data.csv") as infile:
    reader = csv.reader(infile)
    